In [ ]:
%matplotlib inline
# ---- settings ----
import json
from pprint import pprint
Settings = json.load(open('../settings.txt'))
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys
sys.path.insert(0, '../'); sys.path.insert(0, '.')
import paf_loader
from os.path import join, isdir, isfile
import numpy as np
import cv2
from cselect import color as cs
from time import time
from mvpose import pose
from mvpose.algorithm import graphcut
from mvpose.algorithm.settings import get_settings

from mvpose.data import cmu_panoptic

root = Settings['CMU']['data_root']

pprint(Settings['CMU'])
    
FRAME = 2000

panels = Settings['CMU']['panels']
nodes = Settings['CMU']['nodes']

seq_name = Settings['CMU']['seq_name']

tmp = Settings['tmp']

settings = get_settings(max_epi_distance=20, scale_to_mm=10)

# **************************** #
FRAMES = [170, 185, 200, 215, 230, 245, 260, 275, 290, 305]
# **************************** #

def get_track(FRAMES):
    Ims = []
    Hms = []
    Pafs = []
    
    im_name = 'cmu_f' + '_'.join([str(s) for s in FRAMES]) + 'im.npy'
    hm_name = 'cmu_f' + '_'.join([str(s) for s in FRAMES]) + 'hm.npy'
    pf_name = 'cmu_f' + '_'.join([str(s) for s in FRAMES]) + 'pf.npy'
    im_name = join(tmp, im_name)
    hm_name = join(tmp, hm_name)
    pf_name = join(tmp, pf_name)
    
    if isfile(im_name) and isfile(hm_name) and isfile(pf_name):
        Ims = np.load(im_name)
        Hms = np.load(hm_name)
        Pafs = np.load(pf_name)
    else:
        for frame in FRAMES:
            print('load frame ', frame)
            
            
            Im, Y, Calib = cmu_panoptic.get(root, seq_name, 
                                            panels, nodes, 
                                            frame=frame)
            Ims.append(Im)
            heatmaps, pafs = paf_loader.load_confidence_map_and_paf(
                'cmu', Im, frame, dir=tmp)
            Hms.append(heatmaps)
            Pafs.append(pafs)

        Ims = np.array(Ims)
        Hms = np.array(Hms)
        Pafs = np.array(Pafs)
        np.save(im_name, Ims)
        np.save(hm_name, Hms)
        np.save(pf_name, Pafs)
        
    return Ims, Hms, Pafs

_start = time()
Ims, Hms, Pafs = get_track(FRAMES)
_end = time()
print('elapsed', _end - _start)

fig = plt.figure(figsize=(10,12))
n = len(FRAMES)
for frame, Im in enumerate(Ims):
    for cid in range(5):
        ax = fig.add_subplot(n, 5, frame*5+cid+1)
        ax.axis('off')
        ax.imshow(Im[cid])

plt.show()

/home/julian/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


{'data_root': '/storage/DATA/cmu_panoptic',
 'nodes': [1, 15, 3, 13, 8],
 'panels': [1, 6, 14, 18, 5],
 'seq_name': '160422_ultimatum1'}
load frame  170
load frame  185
load frame  200
load frame  215
load frame  230
